In [1]:
from gezi.common import *
sys.path.append('..')
from src.config import *
from src.preprocess import *
from src.eval import *
gezi.init_flags()

In [163]:
root = '../working/offline/6/0'
context_model_name = 'deberta-v3-small.flag-context2-aug.n_context-40.cls_loss_rate-0.1.eval.p13-4'
pairwise_model_name = 'all-mpnet-base-v2.flag-pairwise13-4.pooling_mask-attention_mask.grad_acc-4'
pmodel2_name = 'deberta-v3-small.flag-pairwise14-2-cat.eval'

In [3]:
xc = gezi.load(f'{root}/{context_model_name}/valid.pkl')

In [4]:
xp = gezi.load(f'{root}/{pairwise_model_name}/valid.pkl')

In [202]:
xp2 = gezi.load(f'{root}/{pmodel2_name}/valid.pkl')

In [203]:
gezi.sort_dict_byid_(xc, 'cid')
gezi.sort_dict_byid_(xp, 'cid')
gezi.sort_dict_byid_(xp2, 'cid')

In [201]:
xc['cls_pred_ori'].shape

(424943, 100)

In [204]:
xp2['cls_pred_ori'].shape

(424943, 2)

In [6]:
ids = set(xc['id'])

In [7]:
df_gt = pd.read_csv(f'{FLAGS.root}/train_orders.csv')
df_gt = df_gt[df_gt.id.isin(ids)]
df_gt['cell_order'] = df_gt['cell_order'].apply(lambda x: x.split())
df_gt.head()

,id,cell_order
4,0002115f48f982,"[9ec225f0, 18281c6c, e3b6b115, 4a044c54, 365fe..."
11,00062ab8487156,"[dcad687f, a2e1fc80, 7d977ee8, 45a82a59, cbbc3..."
28,000efd285fb982,"[74a30f80, ee2c8e08, 5523374e, ae8f8fe8, 2138e..."
39,0012865b766949,"[f9cb50e9, 25f7db90, d804e819, 6593a545, fc5bb..."
42,001308991e0c5e,"[21147235, 6c01d0d2, 5bd28595, b8fd3a8c, a2501..."


In [8]:
def merge(x, y, prob):
  # return y
  # return x
  if prob > 0.9:
    return x * (1 - 0.0001) + y * 0.0001
  elif abs(y - x) < 0.1:
    return x * (1 - 0.0001) + y * 0.0001
  elif abs(y - x) < 0.2:
    return x * 0.95 * prob + y * (1 - 0.95 * prob)
  elif abs(y - x) < 0.3:
    return x * 0.85 * prob + y * (1 - 0.85 * prob)
  elif abs(y - x) < 0.4:
    return x * 0.5 * prob + y * (1 - 0.5 * prob)
  else:
    return y

In [9]:
x = xc.copy()
x['pred'] = xc['cls_pred'] * 0.5 + xp['cls_pred'] * 0.5
calc_metric(x)

0.9018247872358709

In [10]:
df_p = pd.DataFrame(xp)

In [11]:
df_c = pd.DataFrame(gezi.batch2list(xc))

In [12]:
df_p.columns

Index(['id', 'cell_id', 'cid', 'pred', 'max_prob', 'max_sim', 'cls_pred',
       'match_rank', 'match_code', 'n_code_cell', 'probs', 'sims'],
      dtype='object')

In [13]:
df = df_c.merge(df_p[['cid', 'pred', 'cls_pred', 'max_prob', 'max_sim', 'probs', 'sims']], on='cid', suffixes=['_c', '_p'])

In [14]:
df_train = pd.read_feather('../working/train.fea')

In [15]:
df_train = df_train[df_train.id.isin(ids)]

In [16]:
df = df.merge(df_train[['cid', 'ancestor_id', 'n_words', 'source']], on='cid')

In [17]:
gezi.set_fold(df, 5, 'ancestor_id')

In [18]:
df.head()

,id,cell_id,cid,cell_type,rank,rel_rank,context_match,match_code,n_code_cell,n_markdown_cell,...,pred_p,cls_pred_p,max_prob,max_sim,probs,sims,ancestor_id,n_words,source,fold
0,0002115f48f982,9ec225f0,0002115f48f982\t9ec225f0,markdown,0,0.055556,1,0.0,8,1,...,0.055556,0.057101,0.990312,0.812567,"[0.99031216, 0.0077258605, 0.00028545051, 0.00...","[0.812567, 0.71245015, 0.6444137, 0.67712796, ...",272b483a,49,"Hi there,\n\nIs it ok that the same measuremen...",2
1,00062ab8487156,96c8449c,00062ab8487156\t96c8449c,markdown,11,0.954545,1,10.0,10,2,...,0.954545,0.928870,0.737295,0.778814,"[0.0006601951, 0.0004797048, 0.0005591548, 3.8...","[0.634043, 0.6274551, 0.6306165, 0.57551205, 0...",de362dd8,9,### Support vector classifier scored 0.8442 on...,1
2,00062ab8487156,aa354742,00062ab8487156\taa354742,markdown,5,0.500000,1,5.0,10,2,...,0.500000,0.470898,0.668899,0.644071,"[0.00014830093, 0.0025996333, 0.08719555, 0.04...","[0.47050405, 0.5295802, 0.60204184, 0.58852756...",de362dd8,40,### Interesting tidbit: Shovel Knight is the o...,1
3,000efd285fb982,14f36391,000efd285fb982\t14f36391,markdown,56,0.881579,1,33.0,37,28,...,0.907895,0.906050,0.779621,0.630970,"[8.096203e-06, 7.811426e-08, 7.105523e-08, 1.2...","[0.39426038, 0.29852632, 0.29657254, 0.3077618...",389f14ff,3,### 3.5 CatboostRegressor,4
4,000efd285fb982,1f3749ad,000efd285fb982\t1f3749ad,markdown,24,0.335526,1,12.0,37,28,...,0.328947,0.327677,0.965926,0.663878,"[6.5797317e-06, 7.1168693e-06, 3.997141e-05, 3...","[0.4184703, 0.42008907, 0.4556867, 0.4516977, ...",389f14ff,4,### 2.1 Numerical columns,4


In [208]:
df['markdown_frac'] = df['n_markdown_cell'] / df['n_cell']
df['span'] = 1 / (df['n_code_cell'] + 1)
top2, top3, top4, top5 = [], [], [], []
top2_prob, top3_prob, top4_prob, top5_prob = [], [], [], []
top2_sim, top3_sim, top4_sim, top5_sim = [], [], [], []
for i in tqdm(range(len(df)), desc='top'):
  # cls_preds = df['cls_pred_ori'].values[i]
  n_code = df['n_code_cell'].values[i]
  probs = df['probs'].values[i]
  sims = df['sims'].values[i]
  idxes = (-probs).argsort()
  if len(idxes) > 1:
    top2.append((idxes[1] + 0.5) / (n_code + 1))
    top2_prob.append(probs[idxes[1]])
    top2_sim.append(sims[idxes[1]])
  else:
    top2.append(-1)
    top2_prob.append(-1)
    top2_sim.append(-1)
  if len(idxes) > 2:
    top3.append((idxes[2] + 0.5) / (n_code + 1))
    top3_prob.append(probs[idxes[2]])
    top3_sim.append(sims[idxes[2]])
  else:
    top3.append(-1)
    top3_prob.append(-1)
    top3_sim.append(-1)
  if len(idxes) > 3:
    top4.append((idxes[3] + 0.5) / (n_code + 1))
    top4_prob.append(probs[idxes[3]])
    top4_sim.append(sims[idxes[3]])
  else:
    top4.append(-1)
    top4_prob.append(-1)
    top4_sim.append(-1)
  if len(idxes) > 4:
    top5.append((idxes[4] + 0.5) / (n_code + 1))
    top5_prob.append(probs[idxes[4]])
    top5_sim.append(sims[idxes[4]])
  else:
    top5.append(-1)
    top5_prob.append(-1)
    top5_sim.append(-1)
ctop_prob, ctop2, ctop3, ctop4, ctop2_prob, ctop3_prob, ctop4_prob = [], [], [], [], [], [], []
for i in tqdm(range(len(df)), desc='ctop'):
  preds = df['cls_pred_ori'].values[i]
  probs = gezi.softmax(preds)
  idxes = (-probs).argsort()
  ctop_prob.append(probs[idxes[0]])
  ctop2.append((idxes[1] + 0.5) / FLAGS.num_classes)
  ctop2_prob.append(probs[idxes[1]])
  ctop3.append((idxes[2] + 0.5) / FLAGS.num_classes)
  ctop3_prob.append(probs[idxes[2]])
  ctop4.append((idxes[3] + 0.5) / FLAGS.num_classes)
  ctop4_prob.append(probs[idxes[3]])
# for i in range(FLAGS.num_classes):
#   df[f'cls_pred{i}'] = df['cls_pred_ori'].apply(lambda x: x[i])
df['top2'] = top2
df['top2_prob'] = top2_prob
df['top2_sim'] = top2_sim
df['top3'] = top3
df['top3_prob'] = top3_prob
df['top3_sim'] = top3_sim
df['top4'] = top4
df['top4_prob'] = top4_prob
df['top4_sim'] = top4_sim
df['top5'] = top5
df['top5_prob'] = top5_prob
df['top5_sim'] = top5_sim
df['ctop_prob'] = ctop_prob
df['ctop2'] = ctop2
df['ctop2_prob'] = ctop2_prob
df['ctop3'] = ctop3
df['ctop3_prob'] = ctop3_prob
df['ctop4'] = ctop4
df['ctop4_prob'] = ctop4_prob
df['pred_p2'] = xp2['pred']
p2_max_prob = []
p2_top2 = []
p2_top2_prob = []
for i in tqdm(range(len(df)), desc='p2'):
  code_idxes = xp2['code_idxes'][i]
  n_code = df['n_code_cell'].values[i]
  probs = gezi.softmax(xp2['cls_pred_ori'][i])
  idxes = (-probs).argsort()
  p2_top2.append((code_idxes[idxes[1]] + 0.5) / (n_code + 1))
  p2_max_prob.append(probs[idxes[0]])
  p2_top2_prob.append(probs[idxes[1]])
df['p2_max_prob'] = p2_max_prob
df['p2_top2'] = p2_top2
df['p2_top2_prob'] = p2_top2_prob
  
df['pred_diff0'] = abs(df['pred_c'] - df['pred_p'])
df['pred_diff1'] = abs(df['reg_pred'] - df['pred_p'])
df['pred_diff2'] = abs(df['cls_pred_c'] - df['pred_p'])
df['pred_diff3'] = abs(df['cls2_pred'] - df['pred_p'])
df['pred_diff4'] = abs(df['pred_c'] - df['top2'])
df['pred_diff5'] = abs(df['reg_pred'] - df['top2'])
df['pred_diff6'] = abs(df['cls_pred_c'] - df['top2'])
df['pred_diff7'] = abs(df['cls2_pred'] - df['top2'])
df['pred_diff8'] = abs(df['pred_p'] - df['top2'])
df['pred_diff9'] = abs(df['cls_pred_c'] - df['top3'])
df['pred_diff10'] = abs(df['pred_p'] - df['top3'])
df['pred_diff11'] = abs(df['pred_c'] - df['pred_p2'])
df['pred_diff12'] = abs(df['reg_pred'] - df['pred_p2'])
df['pred_diff13'] = abs(df['cls_pred_c'] - df['pred_p2'])
df['rule_pred'] = [merge(x, y, prob) for x, y, prob in tqdm(zip(df.pred_p.values, df.pred_c.values, df.max_prob.values), total=len(df), desc='rule')]
# reg_pred_probs = []
# for row in tqdm(df.itertuples(), total=len(df), desc='cls_pred_p_prob'):
#   probs = row.probs
#   n_code = row.n_code_cell
#   reg_pred = row.reg_pred
#   try:
#     reg_pred_probs.append(probs[int(min(reg_pred, 1) * (n_code + 1) - 0.5)])
#   except Exception:
#     ic(n_code, len(probs), reg_pred)
#     break
# df['reg_pred_prob'] = reg_pred_probs

top:   0%|          | 0/424943 [00:00<?, ?it/s]

ctop:   0%|          | 0/424943 [00:00<?, ?it/s]

p2:   0%|          | 0/424943 [00:00<?, ?it/s]

rule:   0%|          | 0/424943 [00:00<?, ?it/s]

In [213]:
fold = 0
dvalid = df[df.fold==fold]
dtrain = df[df.fold!=fold]

In [214]:
df.columns

Index(['id', 'cell_id', 'cid', 'cell_type', 'rank', 'rel_rank',
       'context_match', 'match_code', 'n_code_cell', 'n_markdown_cell',
       'n_cell', 'cls_pred_c', 'pred_c', 'reg_pred', 'cls_pred_ori',
       'match_rank', 'cls2_pred', 'pred_p', 'cls_pred_p', 'max_prob',
       'max_sim', 'probs', 'sims', 'ancestor_id', 'n_words', 'source', 'fold',
       'pred_diff0', 'pred_diff1', 'pred_diff2', 'pred_diff3', 'markdown_frac',
       'span', 'top2', 'top2_prob', 'top2_sim', 'top3', 'top3_prob',
       'top3_sim', 'top4', 'top4_prob', 'top4_sim', 'top5', 'top5_prob',
       'top5_sim', 'ctop_prob', 'ctop2', 'ctop2_prob', 'ctop3', 'ctop3_prob',
       'ctop4', 'ctop4_prob', 'pred_diff4', 'pred_diff5', 'pred_diff6',
       'pred_diff7', 'pred_diff8', 'pred_diff9', 'pred_diff10', 'rule_pred',
       'p2_pred', 'pred_p2', 'pred_diff11', 'pred_diff12', 'pred_diff13',
       'p2_max_prob', 'p2_top2', 'p2_top2_prob'],
      dtype='object')

In [215]:
df

,id,cell_id,cid,cell_type,rank,rel_rank,context_match,match_code,n_code_cell,n_markdown_cell,...,pred_diff10,rule_pred,p2_pred,pred_p2,pred_diff11,pred_diff12,pred_diff13,p2_max_prob,p2_top2,p2_top2_prob
0,0002115f48f982,9ec225f0,0002115f48f982\t9ec225f0,markdown,0,0.055556,1,0.0,8,1,...,0.333333,0.055556,0.055556,0.055556,0.004312,0.003802,0.004823,0.987270,0.166667,0.012730
1,00062ab8487156,96c8449c,00062ab8487156\t96c8449c,markdown,11,0.954545,1,10.0,10,2,...,0.181818,0.954545,0.954545,0.954545,0.007774,0.007617,0.023164,0.574132,0.863636,0.425868
2,00062ab8487156,aa354742,00062ab8487156\taa354742,markdown,5,0.500000,1,5.0,10,2,...,0.272727,0.499997,0.500000,0.500000,0.034678,0.023598,0.045759,0.955423,0.409091,0.044577
3,000efd285fb982,14f36391,000efd285fb982\t14f36391,markdown,56,0.881579,1,33.0,37,28,...,0.026316,0.907890,0.881579,0.881579,0.018504,0.014635,0.022374,0.979487,0.907895,0.020513
4,000efd285fb982,1f3749ad,000efd285fb982\t1f3749ad,markdown,24,0.335526,1,12.0,37,28,...,0.026316,0.328948,0.328947,0.328947,0.006796,0.006120,0.007471,0.910752,0.276316,0.089248
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
424938,fff4714a37cf49,9527a079,fff4714a37cf49\t9527a079,markdown,10,0.500000,1,6.0,12,9,...,0.384615,0.499998,0.500000,0.500000,0.022804,0.020201,0.025407,0.996183,0.576923,0.003817
424939,fff4714a37cf49,effc5f17,fff4714a37cf49\teffc5f17,markdown,6,0.346154,1,4.0,12,9,...,0.076923,0.346156,0.346154,0.346154,0.021974,0.001143,0.045092,0.810956,0.576923,0.189044
424940,fff4714a37cf49,f8744de1,fff4714a37cf49\tf8744de1,markdown,17,0.807692,1,10.0,12,9,...,0.307692,0.807693,0.807692,0.807692,0.006795,0.012041,0.001549,0.976145,0.115385,0.023855
424941,fffc1b4f08e81c,143cccfb,fffc1b4f08e81c\t143cccfb,markdown,29,0.730769,1,28.0,38,2,...,0.128205,0.650635,0.884615,0.884615,0.111127,0.114907,0.107348,0.549838,0.602564,0.450162


In [228]:
reg_cols =  [
          'n_code_cell',
          'n_markdown_cell',
          'n_cell',
          'cls_pred_c',
          'pred_c',
          'reg_pred',
          'cls2_pred',
          'pred_p',
          'cls_pred_p',
          'rule_pred',
          'pred_diff0',
          'pred_diff1',
          'pred_diff2',
          'pred_diff3',
          'pred_diff4',
          'pred_diff5',
          'pred_diff6',
          'pred_diff7',
          'pred_diff8',
          'pred_diff9',
          'pred_diff10',
          'pred_diff11',
          'pred_diff12',
          'pred_diff13',
          'max_sim',
          'max_prob',
          'markdown_frac',
          'span',
          'top2',
          'top2_prob',
          'top2_sim',
          'top3',
          'top3_prob',
          'top3_sim',
          'top4',
          'top4_prob',
          'top4_sim',
          'top5',
          'top5_prob',
          'top5_sim',
          'ctop_prob', 
          'ctop2', 
          'ctop3', 
          'ctop4', 
          'ctop2_prob', 
          'ctop3_prob', 
          'ctop4_prob',
          'pred_p2',
          'p2_max_prob',
          'p2_top2',
          'p2_top2_prob',
        ]
# for i in range(FLAGS.num_classes):
#   reg_cols += [f'cls_pred{i}']
cat_cols = [
          
            ]
label_col = 'rel_rank'
cols = reg_cols + cat_cols
X_train = dtrain[cols]
y_train = dtrain[label_col]
X_valid = dvalid[cols]
y_valid = dvalid[label_col]

In [229]:
from catboost import CatBoostRegressor
xgb_params = {'learning_rate': 0.02,
              'reg_lambda': 7.960622217848342e-07, 
              'subsample': 0.7422597612762745,
              # 'bagging_temperature': 0.2,
              'max_depth': 10, 
              'early_stopping_rounds': 500,
              'n_estimators': 10000,
              'cat_features': [],
              'loss_function': 'MAE',
              }

xgb_params2 = {'learning_rate': 0.09827605967564293,'tree_method':'gpu_hist', 'gpu_id':0,
               'early_stopping_rounds': 50,
               'n_estimators': 10000, }

In [230]:
model = CatBoostRegressor(**xgb_params)
model.fit(X_train, y_train,
        eval_set=[(X_train, y_train), (X_valid, y_valid)],
                verbose=10,
                )  

0:	learn: 0.2544626	test: 0.2544626	test1: 0.2543977	best: 0.2543977 (0)	total: 75ms	remaining: 12m 29s
10:	learn: 0.2165621	test: 0.2165621	test1: 0.2166725	best: 0.2166725 (10)	total: 782ms	remaining: 11m 50s
20:	learn: 0.1859191	test: 0.1859191	test1: 0.1862222	best: 0.1862222 (20)	total: 1.57s	remaining: 12m 27s
30:	learn: 0.1614132	test: 0.1614132	test1: 0.1618599	best: 0.1618599 (30)	total: 2.45s	remaining: 13m 6s
40:	learn: 0.1416753	test: 0.1416753	test1: 0.1422601	best: 0.1422601 (40)	total: 3.22s	remaining: 13m 1s
50:	learn: 0.1257358	test: 0.1257358	test1: 0.1264670	best: 0.1264670 (50)	total: 4.05s	remaining: 13m 9s
60:	learn: 0.1131679	test: 0.1131679	test1: 0.1140440	best: 0.1140440 (60)	total: 4.91s	remaining: 13m 19s
70:	learn: 0.1031169	test: 0.1031169	test1: 0.1041295	best: 0.1041295 (70)	total: 5.7s	remaining: 13m 17s
80:	learn: 0.0951087	test: 0.0951087	test1: 0.0962393	best: 0.0962393 (80)	total: 6.46s	remaining: 13m 11s
90:	learn: 0.0887332	test: 0.0887332	test1: 

In [231]:
dvalid['cb_pred'] = model.predict(dvalid[cols])

In [232]:
ic(calc_metric({'id': dvalid.id.values, 'cell_id': dvalid.cell_id.values, 'pred': dvalid.cb_pred.values}))

[07/19/22 16:33:38] 2876217204.py:1 in <module>
                    calc_metric({'id': dvalid.id.values, 'cell_id': dvalid.cell_id.values, 'pred': dvalid.cb_pred.values}): 0.9110552050830005


0.9110552050830005

In [249]:
import lightgbm as lgb
params = {
          'boosting': 'gbdt',
          'objective': 'regression_l1',
          'metric': {'l1'},
          'num_leaves': 32,
          'min_data_in_leaf': 300,
          # 'max_depth': 10,
          'learning_rate': 0.02,
          "feature_fraction": 0.8,
          "bagging_fraction": 0.75,
          'min_data_in_bin':100,
          "lambda_l1": 1e-7,
          'lambda_l2': 1e-7,
          "random_state": 1024,
          "num_threads": 12,
          }

d_train = lgb.Dataset(X_train, y_train)
d_valid = lgb.Dataset(X_valid, y_valid, reference=d_train)

bst = lgb.train(params,
                d_train,
                100000,
                valid_sets=[d_train, d_valid],
                verbose_eval=1000,
                early_stopping_rounds=500)
dvalid['lgb_pred'] = bst.predict(dvalid[cols])
calc_metric({'id': dvalid.id.values, 'cell_id': dvalid.cell_id.values, 'pred': dvalid.lgb_pred.values})

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.023474 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 12117
[LightGBM] [Info] Number of data points in the train set: 342411, number of used features: 51
[LightGBM] [Info] Start training from score 0.455056
Training until validation scores don't improve for 500 rounds
[1000]	training's l1: 0.0627585	valid_1's l1: 0.0644039
[2000]	training's l1: 0.0617093	valid_1's l1: 0.0637891
[3000]	training's l1: 0.0610143	valid_1's l1: 0.0634572
[4000]	training's l1: 0.0604758	valid_1's l1: 0.0632593
[5000]	training's l1: 0.0601138	valid_1's l1: 0.0631522
[6000]	training's l1: 0.0597749	valid_1's l1: 0.0630649
[7000]	training's l1: 0.0595164	valid_1's l1: 0.0630081
[8000]	training's l1: 0.0592498	valid_1's l1: 0.0629581
[9000]	training's l1: 0.059069	valid_1's l1: 0.0629308
[10000]	training's l1: 0.058902	valid_1's l1: 0.0629089
[11000]	training's l1: 0.058747	

0.9116340034446425

In [250]:
gezi.plot.feature_importance(model, topn=20)

In [251]:
gezi.plot.bar_sorted(np.asarray(bst.feature_importance()), np.asarray(bst.feature_name()),  topn=20)

In [237]:
calc_metric({'id': dvalid.id.values, 'cell_id': dvalid.cell_id.values, 'pred': dvalid.lgb_pred.values})

0.9114947248763512

In [238]:
calc_metric({'id': dvalid.id.values, 'cell_id': dvalid.cell_id.values, 'pred': dvalid.cb_pred.values})

0.9110552050830005

In [239]:
calc_metric({'id': dvalid.id.values, 'cell_id': dvalid.cell_id.values, 'pred': (dvalid.cb_pred.values + dvalid.lgb_pred.values) / 2.})

0.911691966681147

In [240]:
calc_metric({'id': dvalid.id.values, 'cell_id': dvalid.cell_id.values, 'pred': (dvalid.cb_pred.values * 0.3 + dvalid.lgb_pred.values * 0.7)})

0.9117161527798324

In [254]:
df.max_prob.describe(PERCENTILES2)

count    424943.000000
mean          0.732940
std           0.248605
min           0.035726
25%           0.528713
50%           0.800491
75%           0.970395
90%           0.996345
95%           0.998972
99%           0.999920
99.9%         0.999996
max           1.000000
Name: max_prob, dtype: float64

In [255]:
df.top2_prob.describe(PERCENTILES2)

count    4.249430e+05
mean     1.308395e-01
std      1.213924e-01
min      2.737897e-09
25%      1.753494e-02
50%      1.037667e-01
75%      2.147869e-01
90%      3.114589e-01
95%      3.653492e-01
99%      4.454674e-01
99.9%    4.868749e-01
max      4.996785e-01
Name: top2_prob, dtype: float64

In [258]:
len(df[df.fold==0].id.unique())

5391

In [260]:
ids = gezi.random_sample(df[df.fold==0].id.unique(), 332)
ids

array(['60de206b6ce82a', '63f08d7ba7e67d', '26fc66ff507979',
       'eac0f44510c940', 'b08618d68d8944', '5a43e808cd4eeb',
       'fbc01212540d96', 'c3f79b88177999', '50b373eff030e9',
       'fb493afc9137f2', '1756fd2f2bcaeb', '5ca06a49134e94',
       'b5bbeb565abdb2', '6e65077d95d291', '072988fea7f24f',
       '89043fc749b454', 'fc801d9483c98a', '9b16410a01bc18',
       'ae5183a93c12f5', '7c74be67b7cb26', '8b2de0840b86f3',
       '8a7c3e73a0ea8f', 'ae5aebd1d2bdb6', 'fa185cbcafee21',
       '6764a50c754b41', '95afa173a1bada', 'd58edc1ff4868b',
       'a4940a5f2bdec8', 'ba7cd6485d0a30', '66d3a933504cfe',
       '94b3f452bde56a', '7813881d2cac01', '4766c2a18ec9f6',
       'b654321ad2729c', 'afead6eaec311e', 'ef5defb001c002',
       'dc0e2a79cdf89c', '9bc00ac955adc0', '797f683c0b51f3',
       '77a00e97dc9625', '1f2aecf52bd7fa', '12bb5c8ce06644',
       'dd8b72fdb2c201', '88a21b32f1fe78', '061a42cc62fd25',
       '9bdfa8da0dfd36', 'c684ce09fb95ac', '0ab18728d1ad59',
       '08cf0736eb80f3',

In [262]:
d = df[df.id.isin(ids)]

In [263]:
d

,id,cell_id,cid,cell_type,rank,rel_rank,context_match,match_code,n_code_cell,n_markdown_cell,...,pred_diff10,rule_pred,p2_pred,pred_p2,pred_diff11,pred_diff12,pred_diff13,p2_max_prob,p2_top2,p2_top2_prob
2637,01920f87af0b35,2a9f0431,01920f87af0b35\t2a9f0431,markdown,17,0.579710,1,13.0,22,8,...,0.130435,0.586951,0.586957,0.586957,0.051070,0.039388,0.062753,0.983913,0.326087,0.016087
2638,01920f87af0b35,510e6b78,01920f87af0b35\t510e6b78,markdown,0,0.021739,1,0.0,22,8,...,0.043478,0.065217,0.065217,0.065217,0.001709,0.001017,0.004435,0.985653,0.021739,0.014347
2639,01920f87af0b35,5aadfc4c,01920f87af0b35\t5aadfc4c,markdown,3,0.108696,1,2.0,22,8,...,0.043478,0.108695,0.108696,0.108696,0.004336,0.012463,0.003791,0.909259,0.195652,0.090741
2640,01920f87af0b35,71c857a7,01920f87af0b35\t71c857a7,markdown,26,0.847826,1,19.0,22,8,...,0.086957,0.891301,0.891304,0.891304,0.032901,0.030692,0.035110,0.884465,0.847826,0.115535
2641,01920f87af0b35,8a4b0f48,01920f87af0b35\t8a4b0f48,markdown,18,0.594203,1,13.0,22,8,...,0.086957,0.586956,0.586957,0.586957,0.001652,0.002298,0.001006,0.979733,0.500000,0.020267
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
424582,ffdd6d6ae5b35a,7de5fef1,ffdd6d6ae5b35a\t7de5fef1,markdown,34,0.900000,1,22.0,24,14,...,0.240000,0.900000,0.900000,0.900000,0.004710,0.004281,0.005139,0.995273,0.940000,0.004727
424583,ffdd6d6ae5b35a,814a5e21,ffdd6d6ae5b35a\t814a5e21,markdown,9,0.220000,1,5.0,24,14,...,0.080000,0.220001,0.220000,0.220000,0.011718,0.015156,0.008281,0.953805,0.260000,0.046195
424584,ffdd6d6ae5b35a,b7ad7c3a,ffdd6d6ae5b35a\tb7ad7c3a,markdown,5,0.140000,1,3.0,24,14,...,0.040000,0.140000,0.140000,0.140000,0.004457,0.003929,0.004985,0.962857,0.100000,0.037142
424585,ffdd6d6ae5b35a,bde241ff,ffdd6d6ae5b35a\tbde241ff,markdown,13,0.340000,1,8.0,24,14,...,0.360000,0.340000,0.340000,0.340000,0.002559,0.000819,0.004299,0.999990,0.660000,0.000010


In [264]:
calc_metric({'id': d.id.values, 'cell_id': d.cell_id.values, 'pred': d.p2_pred.values})

0.9134030629792227

In [265]:
calc_metric({'id': d.id.values, 'cell_id': d.cell_id.values, 'pred': d.pred_c.values})

0.9141516114022132

In [266]:
calc_metric({'id': d.id.values, 'cell_id': d.cell_id.values, 'pred': d.pred_p.values})

0.9088672641566692

In [267]:
calc_metric({'id': d.id.values, 'cell_id': d.cell_id.values, 'pred': d.cls_pred_p.values})

0.91144065224868